# 20 - NOAA HURDAT2

Download Atlantic best-track text and parse to CSV for quick exposure features.


In [ ]:
from pathlib import Path
import pandas as pd, requests

ROOT = Path('/Users/liamguest/LProjects/AURA/AURA')
RAW = ROOT / 'data' / 'raw'; RAW.mkdir(parents=True, exist_ok=True)
INT = ROOT / 'data' / 'interim'; INT.mkdir(parents=True, exist_ok=True)

HURDAT_URL = 'https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2023-070924.txt'
hurdat_path = RAW / 'hurdat2_atlantic.txt'

r = requests.get(HURDAT_URL, timeout=60)
r.raise_for_status()
hurdat_path.write_bytes(r.content)
print('Wrote', hurdat_path)

lines = hurdat_path.read_text().splitlines()
records = []
current_storm = None
for line in lines:
    if line and line[0].isalpha():
        parts = [p.strip() for p in line.split(',')]
        current_storm = {'id': parts[0], 'name': parts[1], 'n': int(parts[2])}
    else:
        parts = [p.strip() for p in line.split(',')]
        if len(parts) >= 8 and current_storm:
            ymdh = parts[0]
            records.append({
                'storm_id': current_storm['id'],
                'storm_name': current_storm['name'],
                'date': ymdh[:8],
                'time': ymdh[8:],
                'record_id': parts[2],
                'status': parts[3],
                'lat': parts[4],
                'lon': parts[5],
                'max_wind_kt': parts[6],
                'min_pres_mb': parts[7]
            })

hurdat_df = pd.DataFrame(records)
out = INT / 'hurdat2_atlantic_parsed.csv'
hurdat_df.to_csv(out, index=False)
print('Parsed', len(hurdat_df), '->', out)
